# Import required libraries

In [11]:
# Python library used for working with arrays.
import numpy as np

# Python library used for working with data sets.
# Is used for analyzing, cleaning, exploring, and manipulating data.
import pandas as pd

# Most of the Matplotlib utilities lies under the pyplot submodule.
import matplotlib.pyplot as plt

# Huge open-source library for computer vision.
import cv2

# Framework for implementing ML & Deep Learning models.
import tensorflow as tf

# Provides many basic operations on images like create thumnails, resize, 
# rotation, convert between different file formats etc.
from PIL import Image

# OS module provides functions for interacting with the operating system,
# such as with the file system.
import os
# Change current working directory
os.chdir('D:/TSR') 

# Split arrays or matrices into random train and test subsets.
from sklearn.model_selection import train_test_split

# Returns a matrix of binary values. It has #rows = length of the input vector,
# #columns = number of classes.
from keras.utils import to_categorical

# A sequential model allows us to create models layer by layer in a step by step fashion.
# load_model loads a model saved via model.save().
from keras.models import Sequential, load_model

# Layers to be used in the model architecture.
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

# Store data & labels in lists

In [12]:
# Initialising list for storing the data (images) and labels.
data = []
labels = []

# In all we have 43 classes (from 0 to 42)
classes = 43

# Returns current working directory of a process.
cur_path = os.getcwd()

In [13]:
classes

43

In [14]:
cur_path

'D:\\TSR'

# Preprocess the images

In [16]:
# Iterating over all the 43 classes (folders of these classes)
for i in range(classes):
    # Storing the path of the current folder of "Train"
    path = os.path.join(cur_path, 'train', str(i))
    
    # Returns the list of all files and directories in the specified path.
    images = os.listdir(path)
    
    # iterating over all the images of the current class 'i'
    for a in images:
        try:
            image = Image.open(path + '\\' + a)
            image = image.resize((30, 30))
            image = np.array(image)
            data.append(image)
            labels.append(i)
        except Exception as e:
            print(e)

# Convert data & labels lists to numpy arrays

In [17]:
data = np.array(data)
labels = np.array(labels)